In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
# Sample file, from Dec 2024
# https://mbta-massdot.opendata.arcgis.com/datasets/924df13d845f4907bb6a6c3ed380d57a/about
filename = "MBTA-Bus-Arrival-Departure-Times_2024-12.csv"


In [ ]:
df = pd.read_csv(filename, low_memory=False)

In [ ]:
print(df.head())
print(df.shape)
print(df.describe)

In [ ]:
# Calculate delay for each station in seconds
df['time_difference'] = pd.to_datetime(df['actual']) - pd.to_datetime(df['scheduled'])
df['delay_seconds'] = df['time_difference'].dt.total_seconds()


In [ ]:
# Calculate headway delay for each station in seconds 
# Headway is the amount of time that has passed since the previous bus stopped at the station
df['headway_delay'] = df['headway'] - df['scheduled_headway']

In [ ]:
print(df.head())


In [ ]:
# Save as a CSV if wanted, optional to check manually
df.to_csv('MBTA-Bus-Arrival-Departure-Times_2024-12-with-delay.csv', index=False)

In [ ]:
# Understanding where data is missing, what gaps there might be
df['delay_seconds'] = pd.to_numeric(df['delay_seconds'], errors='coerce')
total_count = df.shape[0]
incomplete_count = df['delay_seconds'].isna().sum()

print("Number of total results in delay_seconds:", total_count)
print("Number of incomplete results in delay_seconds:", incomplete_count)
print("Percent incomplete:", incomplete_count/total_count)

In [ ]:
df['delay_headway'] = pd.to_numeric(df['delay_headway'], errors='coerce')
total_count = df.shape[0]

incomplete_count = df['delay_headway'].isna().sum()

print("Number of total results in delay_headway:", total_count)
print("Number of incomplete results in delay_headway:", incomplete_count)
print("Percent incomplete:", incomplete_count/total_count)

In [ ]:
# Shows that some routes don't calculate headway, might be a result of station location / if it's an endpoint, need to look into more

grouped = df.groupby(['route_id'])

result = grouped['delay_headway'].agg(
    total_count='size',  
    incomplete_count=lambda x: x.isna().sum() 
)

# Calculate the percent incomplete for each group
result['percent_incomplete'] = result['incomplete_count'] / result['total_count']

for result in result.iterrows():
  print(f"Route: {result[0]}, percent incomplete: {result[1][2]}")

In [ ]:
# Same with some stops, will try grouping by location to see if there's a cause or a way it can be mitigated

grouped = df.groupby(['stop_id'])

result = grouped['delay_headway'].agg(
    total_count='size',  
    incomplete_count=lambda x: x.isna().sum() 
)

# Calculate the percent incomplete for each group
result['percent_incomplete'] = result['incomplete_count'] / result['total_count']

for result in result.iterrows():
  print(f"Stop: {result[0]}, percent incomplete: {result[1][2]}")

In [ ]:
# For now will only look at delay and not headway delay, much more consistent with less missing

delays = df['delay_seconds'].dropna()

counts, bin_edges = np.histogram(delays, bins=1000)
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

plt.figure(figsize=(10, 6))
plt.bar(bin_centers, counts, width=(bin_edges[1] - bin_edges[0]) * 0.9)
plt.title("Delay Distribution for Entire Dataset")
plt.xlabel("Delay (seconds)")
plt.ylabel("Frequency")
plt.show()


# Have issues with outliers, will exclude results over +-1200 seconds (20 minutes) to address miscalculations
# Rough solution for now, can look into further to see if there's an issue with how date is displayed in some cases

In [ ]:
filtered_df = df[(df['delay_seconds'] >= -1200) & (df['delay_seconds'] <= 1200)]
grouped = filtered_df.groupby(['stop_id'])


# Calculate aggregate statistics for each group: average delay, minimum delay, maximum delay.
stats = grouped['delay_seconds'].agg(average_delay='mean',
                                      min_delay='min',
                                      max_delay='max')

print(stats)



# For now ignoring headway delay, very useful but need to look more into what's causing the delay

In [ ]:
# Create histograms of the delays to get a rough understanding of delay distribution for some stops

count = 0

for stop, group in grouped:

    if count % 50 == 0:

      delays = group['delay_seconds'].dropna()
      

      if delays.empty:
          continue


      counts, bin_edges = np.histogram(delays, bins=20)
      

      bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
      

      plt.figure()
      plt.bar(bin_centers, counts, width=(bin_edges[1]-bin_edges[0]) * 0.9)
      plt.title(f"Delay Distribution for Stop {stop}")
      plt.xlabel("Delay (seconds)")
      plt.ylabel("Frequency")
      plt.show()

    count = count + 1 # only want to get a rough understanding, don't need 1000+ diagrams
